In [1]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, log_loss, confusion_matrix 
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import eli5
from eli5.sklearn import PermutationImportance
import xgboost as xgb

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be import

In [2]:
df = pd.read_csv('/content/train.csv')

In [3]:
df['target'].unique()

array(['Class_6', 'Class_2', 'Class_8', 'Class_3', 'Class_1', 'Class_5',
       'Class_7', 'Class_9', 'Class_4'], dtype=object)

In [4]:
df['target_value'] = df['target'].map({'Class_6':5, 'Class_2':1, 'Class_8':7, 'Class_3':2, 'Class_1':0, 'Class_5':4,
       'Class_7':6, 'Class_9':8, 'Class_4':3})

In [5]:
feature_cols = df.columns[1:-2]
X = df[feature_cols]
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

y = df['target']

LE = LabelEncoder()
y = LE.fit_transform(y)

In [6]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=19930609, stratify=y)

scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#performed worse
#oversample = SMOTE(sampling_strategy='minority',random_state=101,k_neighbors=200,n_jobs=-1)
#X_over, y_over = oversample.fit_resample(X_train,y_train)

In [8]:
xgb_model = xgb.XGBClassifier(random_state=42,objective='multi:softmax')
xgb_model.fit(X_train,y_train)
perm = PermutationImportance(xgb_model, random_state=13, scoring = 'neg_log_loss')
perm.fit(X_test,y_test)

PermutationImportance(cv='prefit',
                      estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                              colsample_bylevel=1,
                                              colsample_bynode=1,
                                              colsample_bytree=1, gamma=0,
                                              learning_rate=0.1,
                                              max_delta_step=0, max_depth=3,
                                              min_child_weight=1, missing=None,
                                              n_estimators=100, n_jobs=1,
                                              nthread=None,
                                              objective='multi:softprob',
                                              random_state=42, reg_alpha=0,
                                              reg_lambda=1, scale_pos_weight=1,
                                              seed=None, silent=None,
                     

In [9]:
imp = pd.DataFrame(zip(feature_cols,perm.feature_importances_),columns=['features','importance'])
imp.sort_values(by='importance',inplace=True)
imp

,features,importance
27,feature_27,5.768047e-07
15,feature_15,1.723529e-05
73,feature_73,1.832873e-05
6,feature_6,6.860303e-05
42,feature_42,9.780160e-05
...,...,...
37,feature_37,3.307082e-03
20,feature_20,3.655252e-03
18,feature_18,3.786211e-03
12,feature_12,4.179800e-03


In [10]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=19930609, stratify=y)#

#X_train.drop(['feature_74','feature_36','feature_15'], axis = 1, inplace=True)
#X_test.drop(['feature_74','feature_36','feature_15'], axis = 1, inplace=True)#

#scaler = MinMaxScaler()

#X_train = scaler.fit_transform(X_train)
#X_test = scaler.transform(X_test)

In [11]:
fit_params = {
    'early_stopping_rounds': 5,
    'eval_metric' : 'mlogloss',
    'eval_set': [(X_test, y_test)],
}
xgb_model = xgb.XGBClassifier(base_score=0.5, booster='gbtree', gamma=0.1,
              learning_rate=0.03, max_depth=8, n_estimators=2500, n_jobs=1,
              objective='multi:softprob', random_state=19930609, reg_lambda=20, 
              scale_pos_weight=0.35, seed=19930609, subsample=0.8, verbosity=1)
xgb_model.fit(X_train,y_train,**fit_params)

[0]	validation_0-mlogloss:2.18127
Will train until validation_0-mlogloss hasn't improved in 5 rounds.
[1]	validation_0-mlogloss:2.16607
[2]	validation_0-mlogloss:2.15173
[3]	validation_0-mlogloss:2.13804
[4]	validation_0-mlogloss:2.12497
[5]	validation_0-mlogloss:2.11248
[6]	validation_0-mlogloss:2.10056
[7]	validation_0-mlogloss:2.08916
[8]	validation_0-mlogloss:2.07811
[9]	validation_0-mlogloss:2.06764
[10]	validation_0-mlogloss:2.05761
[11]	validation_0-mlogloss:2.04791
[12]	validation_0-mlogloss:2.03861
[13]	validation_0-mlogloss:2.02967
[14]	validation_0-mlogloss:2.02108
[15]	validation_0-mlogloss:2.01273
[16]	validation_0-mlogloss:2.00473
[17]	validation_0-mlogloss:1.99704
[18]	validation_0-mlogloss:1.9896
[19]	validation_0-mlogloss:1.9824
[20]	validation_0-mlogloss:1.97547
[21]	validation_0-mlogloss:1.96882
[22]	validation_0-mlogloss:1.96233
[23]	validation_0-mlogloss:1.95605
[24]	validation_0-mlogloss:1.95004
[25]	validation_0-mlogloss:1.94422
[26]	validation_0-mlogloss:1.9386


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0.1,
              learning_rate=0.03, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=None, n_estimators=2500, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=19930609,
              reg_alpha=0, reg_lambda=20, scale_pos_weight=0.35, seed=19930609,
              silent=None, subsample=0.8, verbosity=1)

In [12]:
preds = xgb_model.predict(X_test)
cm = confusion_matrix(y_test,preds)
cm

array([[   0,  396,    1,    0,    0,  466,    0,  952,    9],
       [   0, 2375,    0,    0,    0, 1008,    0, 1461,   42],
       [   0, 1109,    1,    0,    0,  677,    0, 1154,   19],
       [   0,  228,    0,    0,    0,  260,    0,  444,    9],
       [   0,  137,    0,    0,    0,  163,    0,  307,    6],
       [   0,  890,    0,    0,    0, 5324,    0, 4106,   42],
       [   0,  329,    2,    0,    0,  790,    0, 1818,   15],
       [   0,  786,    0,    0,    0, 2807,    0, 6728,   31],
       [   0, 1123,    0,    0,    0, 1420,    0, 2528,   37]])

In [13]:
print(accuracy_score(y_test,preds))
print(f1_score(y_test,preds,average='weighted'))

0.361625
0.2843218512869043


In [16]:
final = pd.read_csv('/content/test.csv')
X2=final[final.columns[1:]]

In [17]:
X2 = scaler.transform(X2)

In [18]:
predictions = pd.DataFrame(xgb_model.predict_proba(X2), columns=['Class_1','Class_2','Class_3','Class_4','Class_5','Class_6','Class_7','Class_8','Class_9'])
predictions['id'] = final['id']
predictions.set_index('id',inplace=True)
predictions.to_csv('submission_XGB_final.csv',header=True,index=True)